In [20]:
import os

# 읽은 파일 내용을 저장할 빈 리스트 생성
poker_hand_data = []

# 파일이 저장된 디렉토리 경로 설정
directory_path = 'E:\chatGPT\Holdem\HandData'

# 디렉토리 내의 모든 파일에 대해 반복
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # 파일 확장자에 따라 필터링
        file_path = os.path.join(directory_path, filename)
        
        # 파일을 읽어서 리스트에 추가
        with open(file_path, 'r') as file:
            file_contents = file.read()
            poker_hand_data.append(file_contents)

hand_data_list = []

# 핸드데이터 정리
for poker_hand in poker_hand_data:
    hand_data = [line.strip() for line in poker_hand.strip().split('\n\n')]

    hand_data_list = hand_data_list + hand_data

import pandas as pd
import re

# DataFrame을 저장할 리스트
hero_hands = []

# 각 포커 핸드 데이터에 대한 반복
for hand_data in hand_data_list:
    hand_date = hand_data.split('\n')[0]
    
    # 날짜를 추출하는 정규 표현식 패턴
    date_pattern = r"\d{4}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}"
    
    match = re.search(date_pattern, hand_date)

    FlopCard = []

    if match:
        extracted_date = match.group()
        
    for line in hand_data.split('\n'):
        if "Dealt to Hero" in line:
            hero_hand = line.split('[')[1].split(']')[0]
            # 날짜와 hero_hand 값을 리스트에 추가
        if "*** FLOP ***" in line:
            FlopCard = line.split('[')[1].split(']')[0]
            # Flop 카드 값을 리스트에 추가

    hero_hands.append([extracted_date, hero_hand, FlopCard])

            

# DataFrame 생성
df = pd.DataFrame(hero_hands, columns=['Date', 'Hero Hand', 'FlopCard'])

# 'Date' 열을 datetime 자료형으로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 날짜를 기준으로 DataFrame을 정렬
df = df.sort_values(by='Date')

# 정렬된 DataFrame 출력
Flop = df[df['FlopCard'].apply(len) > 0]

hero_hands = Flop['Hero Hand']
flop_card = Flop['FlopCard']

print(Flop)

                    Date Hero Hand  FlopCard
448  2023-12-21 23:02:10     Jc 8h  Jd 6d As
217  2023-12-21 23:02:25     9c Ah  3d 8c 4d
216  2023-12-21 23:02:30     Ac Ad  9h Ts 4s
442  2023-12-21 23:02:48     4c Kc  9d 4d 3s
214  2023-12-21 23:03:45     Qh 7d  6h 8c 2s
...                  ...       ...       ...
2132 2023-12-27 22:11:46     Jc 2d  Qs 6h 4d
2125 2023-12-27 22:12:52     Jd Kh  3h As 7s
2117 2023-12-27 22:14:03     Th Jh  As Qc Jc
2106 2023-12-27 22:15:43     3d Qh  5s 8s 6h
2105 2023-12-27 22:15:47     As Ad  Qh Qc 4c

[1006 rows x 3 columns]


In [ ]:
import numpy as np

hero_hands = Flop['Hero Hand']
flop_card = Flop['FlopCard']
HoleCards = np.zeros((13, 13), dtype=int)
rank_mapping = {'A': 0, 'K': 1, 'Q': 2, 'J': 3, 'T': 4, '9': 5, '8': 6, '7': 7, '6': 8, '5': 9, '4': 10, '3': 11, '2': 12}

# 홀카드 좌표 인식
for index, row in Flop.iterrows():

    hand = row['Hero Hand']
    cards = hand.split() # cards = ['As', 'Ad']

    flop = row['FlopCard']
    
    
    rank1, rank2 = cards[0][0], cards[1][0]
    suit1, suit2 = cards[0][1], cards[1][1]

    if suit1 == suit2:
        HoleCards[min(rank_mapping[rank1], rank_mapping[rank2]), max(rank_mapping[rank1], rank_mapping[rank2])] += 1
    else:
        HoleCards[max(rank_mapping[rank1], rank_mapping[rank2]), min(rank_mapping[rank1], rank_mapping[rank2])] += 1


HoleCards


In [123]:
'''
                    Date Hero Hand  FlopCard
448  2023-12-21 23:02:10     Jc 8h  Jd 6d As
217  2023-12-21 23:02:25     9c Ah  3d 8c 4d
216  2023-12-21 23:02:30     Ac Ad  9h Ts 4s
'''
Hand = ['3d 5h']
Flop = ['2c Qc 5c']
rank_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}


hand = Hand[0].split()
flop = Flop[0].split()

hand_rank1, hand_rank2 = hand[0][0], hand[1][0]
hand_suit1, hand_suit2 = hand[0][1], hand[1][1]

flop_rank1, flop_rank2, flop_rank3 = flop[0][0], flop[1][0], flop[2][0]
flop_suit1, flop_suit2, flop_suit3 = flop[0][1], flop[1][1], flop[2][1]

ranks = [hand_rank1, hand_rank2, flop_rank1, flop_rank2, flop_rank3]
suits = [hand_suit1, hand_suit2, flop_suit1, flop_suit2, flop_suit3]

##풀하우스 판별
if len(set(ranks)) == 2:
    print('풀하우스입니다.')
    # break

##플러쉬 판별
if len(set(suits)) == 1:
    print('플러쉬입니다.')

##스트레이트 판별
rank_numbers = [rank_values[rank] for rank in ranks]

# rank_numbers 리스트에 A가 포함되어 있으면 1을 리스트에 추가
if 14 in rank_numbers:
    rank_numbers.append(1)
rank_numbers.sort()

# 스트레이트 여부 확인
consecutive_count = 0
for i in range(len(rank_numbers) - 1):
    if rank_numbers[i] + 1 == rank_numbers[i + 1]:
        consecutive_count += 1
        if consecutive_count >= 4:
            print('스트레이트입니다.')
    elif rank_numbers[i] != rank_numbers[i + 1]:
        consecutive_count = 0

##페어 확인

hand_rank = [rank_values[hand_rank1], rank_values[hand_rank2]]
flop_rank = [rank_values[flop_rank1], rank_values[flop_rank2], rank_values[flop_rank3]]

#포켓일 경우
if len(set(hand_rank)) == 1:
    if flop_rank.count(hand_rank[0]) == 0:
        sort = sorted(flop_rank + [hand_rank[0]], reverse=True)
        position = sort.index(hand_rank[0])
        if position == 0:
            print('오버페어')
        elif position == 1:
            print('탑페어')
        elif position == 2:
            print('미들페어')
        elif position == 3:
            print('로우페어')
    elif flop_rank.count(hand_rank[0]) == 1:
        print('셋')
    elif flop_rank.count(hand_rank[0]) == 2:
        print('포카드')

#포켓이 아닐 경우
from collections import Counter

hand_flop_rank = hand_rank + flop_rank

if len(set(hand_rank)) == 2:
    if len(set(flop_rank)) == 3 and len(set(hand_flop_rank)) == 5:
        print('하이카드')
    if len(set(flop_rank)) == 3 and len(set(hand_flop_rank)) == 4:
        onepair = Counter(hand_flop_rank).most_common(1)
        sort = sorted(hand_flop_rank, reverse=True)
        print(sort)
        print(" ")
        position = sort.index(onepair[0][0])
        if position == 0:
            print('탑페어')
        elif position == 1:
            print('미들페어')
        elif position == 2:
            print('로우페어')

'''
Hand = ['3d 5h']
Flop = ['2c Qc 5c']
'''

print(hand_rank)
print(flop_rank)
print(hand_flop_rank)





[12, 5, 5, 3, 2]
 
미들페어
[3, 5]
[2, 12, 5]
[3, 5, 2, 12, 5]


In [37]:
FullHouse = np.zeros((13,13), dtype=int)
Flush = np.zeros((13,13), dtype=int)
Straight = np.zeros((13,13), dtype=int)
Set = np.zeros((13,13), dtype=int)
Trips = np.zeros((13,13), dtype=int)
TwoPair = np.zeros((13,13), dtype=int)
OnePair = np.zeros((13,13), dtype=int)

FlushDraw = np.zeros((13,13), dtype=int)
Gutshot = np.zeros((13,13), dtype=int)
Openended = np.zeros((13,13), dtype=int)

hero_hands = df['Hero Hand']

for hand in hero_hands:
    cards = hand.split()

    